In [1]:
from functools import partial

from transformers import CLIPVisionModel 
import torch
from torch import nn
from torchvision import transforms
from PIL import Image

import torch
import torch.nn as nn
from transformers import CLIPVisionModel
from torchvision import transforms

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')



In [ ]:
train_pixel_img_feature = torch.load('/root/autodl-tmp/Workspace/EEG_caption/ViT-L-14_features_GIT_train.pt')['img_features']# 
test_pixel_img_feature = torch.load('/root/autodl-tmp/Workspace/EEG_caption/ViT-L-14_features_GIT_test.pt')['img_features']# 
train_img_feature = torch.load('/root/autodl-tmp/Workspace/EEG_caption/ViT-H-14_features_train.pt')['img_features'].unsqueeze(1)# 
test_img_feature = torch.load('/root/autodl-tmp/Workspace/EEG_caption/ViT-H-14_features_test.pt')['img_features'].unsqueeze(1)# 


In [ ]:
test_img_feature.shape

In [5]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader, TensorDataset

# # Define the neural network
# class PixelProjector(nn.Module):
#     def __init__(self):
#         super(PixelProjector, self).__init__()
#         self.linear = nn.Linear(1, 256)
        
#     def forward(self, x):
#         # x shape: (batch_size, 1, 1024)
#         batch_size, _, num_features = x.size()
#         x = x.view(-1, 1)  # reshape to (batch_size * num_features, 1)
#         x = self.linear(x)
#         x = x.view(batch_size, 256, num_features)  # reshape to (batch_size, 256, 1024)
#         return x

# # Instantiate the model, loss function, and optimizer

# model = PixelProjector().to(torch.bfloat16).to(device)
# criterion = nn.MSELoss()
# optimizer = optim.AdamW(model.parameters(), lr=0.001)

# # Prepare data loaders
# train_dataset = TensorDataset(train_img_feature, train_pixel_img_feature)
# test_dataset = TensorDataset(test_img_feature, test_pixel_img_feature)

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, drop_last=True)
# test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# # Training loop
# num_epochs = 10
# for epoch in range(num_epochs):
#     model.train()
#     running_loss = 0.0
#     for inputs, targets in train_loader:
#         inputs, targets = inputs.to(torch.bfloat16).to(device), targets.to(torch.bfloat16).to(device)
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = criterion(outputs, targets)
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item()
    
#     print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

# # Testing loop
# model.eval()
# test_loss = 0.0
# with torch.no_grad():
#     for inputs, targets in test_loader:
#         inputs, targets = inputs.to(torch.bfloat16).to(device), targets.to(torch.bfloat16).to(device)
#         outputs = model(inputs)
#         loss = criterion(outputs, targets)
#         test_loss += loss.item()

# print(f"Test Loss: {test_loss/len(test_loader)}")


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from einops.layers.torch import Rearrange, Reduce

# Define the neural network
class PixelProjector(nn.Sequential):
    def __init__(self, proj_dim=1024):
        super().__init__(
            Rearrange('B C L->B L C'),    
            nn.Linear(1, 257),
            nn.LayerNorm(257),
            Rearrange('B L C->B C L'),
            nn.Linear(1024, 1024),
            nn.LayerNorm(proj_dim),
            )
        
        

# Instantiate the model, loss function, and optimizer

model = PixelProjector(proj_dim=1024).to(torch.bfloat16).to(device)
criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)

# Prepare data loaders
train_dataset = TensorDataset(train_img_feature, train_pixel_img_feature)
test_dataset = TensorDataset(test_img_feature, test_pixel_img_feature)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Training loop
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(torch.bfloat16).to(device), targets.to(torch.bfloat16).to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

# Testing loop
model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(torch.bfloat16).to(device), targets.to(torch.bfloat16).to(device)
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()

print(f"Test Loss: {test_loss/len(test_loader)}")

# Save the trained model
torch.save(model.state_dict(), '/root/autodl-tmp/Workspace/EEG_caption/model_weights/PixelProjector_best.bin')
print("Model saved as PixelProjector.bin")


In [8]:
# Save the trained model
torch.save(model.state_dict(), '/root/autodl-tmp/Workspace/EEG_caption/model_weights/PixelProjector_best.bin')
print("Model saved as PixelProjector.bin")

Model saved as PixelProjector.bin


In [ ]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader, TensorDataset
# from einops.layers.torch import Rearrange, Reduce
# from torchvision import transforms

# # Define the neural network
# class PixelProjector(nn.Sequential):
#     def __init__(self, proj_dim=1024):
#         super().__init__(
#             Rearrange('B C L->B L C'),    
#             nn.Linear(1, 256),
#             nn.LayerNorm(256),
#             # nn.Dropout(0.5),  # Add dropout
#             Rearrange('B L C->B C L'),
#             nn.Linear(1024, 1024),
#             nn.LayerNorm(proj_dim),
#             # nn.Dropout(0.5)   # Add dropout
#         )

# # Instantiate the model, loss function, and optimizer
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = PixelProjector(proj_dim=1024).to(torch.bfloat16).to(device)
# criterion = nn.MSELoss()
# optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)  # Add weight decay

# # Data augmentation transforms
# data_augmentation = transforms.Compose([
#     transforms.RandomHorizontalFlip(),
#     # transforms.RandomVerticalFlip(),
#     # transforms.RandomRotation(degrees=15),
#     # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
#     # transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.1)),
# ])

# # Create augmented TensorDataset
# class AugmentedTensorDataset(TensorDataset):
#     def __init__(self, *tensors, transform=None):
#         super().__init__(*tensors)
#         self.transform = transform

#     def __getitem__(self, index):
#         x, y = super().__getitem__(index)
#         if self.transform:
#             x = self.transform(x)
#         return x, y

# # Apply data augmentation to training dataset
# train_dataset = AugmentedTensorDataset(train_img_feature, train_pixel_img_feature, transform=data_augmentation)
# test_dataset = TensorDataset(test_img_feature, test_pixel_img_feature)

# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
# test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# # Training loop with early stopping
# num_epochs = 30
# best_test_loss = float('inf')
# early_stopping_patience = 5
# patience_counter = 0

# for epoch in range(num_epochs):
#     model.train()
#     running_loss = 0.0
#     for inputs, targets in train_loader:
#         inputs, targets = inputs.to(torch.bfloat16).to(device), targets.to(torch.bfloat16).to(device)
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = criterion(outputs, targets)
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item()
    
#     model.eval()
#     test_loss = 0.0
#     with torch.no_grad():
#         for inputs, targets in test_loader:
#             inputs, targets = inputs.to(torch.bfloat16).to(device), targets.to(torch.bfloat16).to(device)
#             outputs = model(inputs)
#             loss = criterion(outputs, targets)
#             test_loss += loss.item()

#     test_loss /= len(test_loader)
#     print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {running_loss/len(train_loader)}, Test Loss: {test_loss}")

#     # Early stopping
#     if test_loss < best_test_loss:
#         best_test_loss = test_loss
#         patience_counter = 0
#         torch.save(model.state_dict(), '/home/ldy/Workspace/BrainAligning/model_weights/PixelProjector_best.bin')
#     else:
#         patience_counter += 1
#         if patience_counter >= early_stopping_patience:
#             print("Early stopping triggered")
#             break

# print(f"Best Test Loss: {best_test_loss}")
# print("Model saved as PixelProjector_best.bin")


In [ ]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader, TensorDataset
# from einops.layers.torch import Rearrange, Reduce
# from torchvision import transforms

# # Define the neural network
# class PixelProjector(nn.Sequential):
#     def __init__(self, proj_dim=1024):
#         super().__init__(
#             Rearrange('B C L -> B L C'),    
#             nn.Linear(1, 256),
#             # nn.LayerNorm(256),
#             nn.Dropout(0.5),  # Add dropout
#             Rearrange('B L C -> B C L'),
#             nn.Linear(1024, 1024),
#             # nn.LayerNorm(proj_dim),
#             nn.Dropout(0.5)   # Add dropout
#         )

# # Instantiate the model, loss function, and optimizer
# device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
# model = PixelProjector(proj_dim=1024).to(torch.bfloat16).to(device)
# criterion = nn.MSELoss()
# optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)  # Adjust learning rate

# # Cosine Annealing Scheduler
# scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=1e-6)

# # Data augmentation transforms
# data_augmentation = transforms.Compose([
#     transforms.RandomHorizontalFlip(),
#     # transforms.RandomVerticalFlip(),
#     # transforms.RandomRotation(degrees=15),
#     # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
#     # transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.1)),
# ])

# # Create augmented TensorDataset
# class AugmentedTensorDataset(TensorDataset):
#     def __init__(self, *tensors, transform=None):
#         super().__init__(*tensors)
#         self.transform = transform

#     def __getitem__(self, index):
#         x, y = super().__getitem__(index)
#         if self.transform:
#             x = self.transform(x)
#         return x, y

# # Apply data augmentation to training dataset
# train_dataset = AugmentedTensorDataset(train_img_feature, train_pixel_img_feature, transform=data_augmentation)
# test_dataset = TensorDataset(test_img_feature, test_pixel_img_feature)

# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
# test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# # Training loop with early stopping
# num_epochs = 30
# best_test_loss = float('inf')
# early_stopping_patience = 5
# patience_counter = 0

# for epoch in range(num_epochs):
#     model.train()
#     running_loss = 0.0
#     for inputs, targets in train_loader:
#         inputs, targets = inputs.to(torch.bfloat16).to(device), targets.to(torch.bfloat16).to(device)
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = criterion(outputs, targets)
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item()
    
#     model.eval()
#     test_loss = 0.0
#     with torch.no_grad():
#         for inputs, targets in test_loader:
#             inputs, targets = inputs.to(torch.bfloat16).to(device), targets.to(torch.bfloat16).to(device)
#             outputs = model(inputs)
#             loss = criterion(outputs, targets)
#             test_loss += loss.item()

#     test_loss /= len(test_loader)
#     print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {running_loss/len(train_loader)}, Test Loss: {test_loss}")

#     # Adjust the learning rate based on the test loss
#     scheduler.step()

#     # Early stopping
#     if test_loss < best_test_loss:
#         best_test_loss = test_loss
#         patience_counter = 0
#         torch.save(model.state_dict(), '/home/ldy/Workspace/BrainAligning/model_weights/PixelProjector_best.bin')
#     else:
#         patience_counter += 1
#         if patience_counter >= early_stopping_patience:
#             print("Early stopping triggered")
#             break

# print(f"Best Test Loss: {best_test_loss}")
# print("Model saved as PixelProjector_best.bin")
